In [1]:
from pathlib import Path
import requests
filename = Path('test.pdf')
url = 'https://app.viac.ch/files/document/21V-L0F-58I/content/21V-L0F-58I.pdf'
response = requests.get(url)
filename.write_bytes(response.content)

0

In [67]:
import json
import csv
import os
import re
from datetime import datetime, timedelta
import fitz  # PyMuPDF

# Load the JSON data from the file
with open('transactions.json', 'r') as file:
    data = json.load(file)
    


# Define a function to extract shares and exchange rate from a PDF file
def extract_shares_and_exchange_rate(document_number):
    shares = ''
    exchange_rate = ''
    isin = ''
    

    # Find the PDF file that contains the document number as a substring
    pdf_folder = 'pdfs'
    pdf_files = [f for f in os.listdir(pdf_folder) if document_number in f]

    if not pdf_files:
        raise FileNotFoundError(f"No PDF file found containing document number {document_number}")

    pdf_path = os.path.join(pdf_folder, pdf_files[0])

    # Open the PDF file
    pdf_document = fitz.open(pdf_path)

    # Iterate through each page
    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)
        text = page.get_text()

        # Search for shares value
        shares_match = re.search(r'(?:Kauf|Buy|Verkauf|Sell)\n(\d+\.\d+)', text)
        if shares_match:
            shares = shares_match.group(1)
        if len(shares)<2:
            print(f"Error in transaction {document_number}. File exists but could not find number of shares!")
        

        currency = ""
        # Search for exchange rate value
        exchange_rate_match = re.search(r'(?:Exchange rate|Umrechnungskurs) [A-Z]{3}/[A-Z]{3} (\d+\.\d+)\n', text)
        if exchange_rate_match:
            exchange_rate = exchange_rate_match.group(1)
            exchange_rate = "{:3.8f}".format(1/float(exchange_rate)) # take the inverse
            
            currency_match = re.search(r'(?:Exchange rate|Umrechnungskurs) [A-Z]{3}/([A-Z]{3})\s*\d+\.\d+', text, re.DOTALL)
            if currency_match:
                currency = currency_match.group(1)
            
            
            
        # Search for ISIN
        isin_match = re.search(r'ISIN.{0,8}([A-Z0-9]{12})', text, re.DOTALL)
        if isin_match:
            isin = isin_match.group(1)
        

    pdf_document.close()

    
    return shares, exchange_rate, isin, currency


last_ex_rate = {}
# Define a function to process transactions and generate CSV for each account
def process_transactions(account_id, transactions, securities):
    # Prepare the CSV file name
    csv_file_name = f"account_{account_id.replace('.', '_')}.csv"
    
    # Open the CSV file for writing
    with open(csv_file_name, 'w', newline='') as csvfile:
        fieldnames = ['Date', 'Type', 'Value', 'Security Name', 'Transaction Currency', 'Shares', 'Exchange Rate', 'Gross Amount', 'Currency Gross Amount']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        # Write the header
        writer.writeheader()
        
        # Track dividend cancellations to ignore corresponding dividends
        dividend_cancellations = []
        
        # First pass: build the list of dividend cancellations
        for transaction in transactions:
            if transaction['type'] == 'DIVIDEND_CANCELLATION':
                dividend_cancellations.append(transaction)
        
        # Second pass: process transactions and write to CSV
        for transaction in transactions[::-1]:
            if transaction['type'] == 'DIVIDEND_CANCELLATION':
                continue
            
            if transaction['type'] == 'DIVIDEND':
                # Check if there is a matching DIVIDEND_CANCELLATION within 30 days
                cancel = False
                for cancel_transaction in dividend_cancellations:
                    cancel_date = datetime.strptime(cancel_transaction['valueDate'], '%Y-%m-%d')
                    transaction_date = datetime.strptime(transaction['valueDate'], '%Y-%m-%d')
                    if (cancel_transaction['amountInChf'] == transaction['amountInChf'] and
                        abs((transaction_date - cancel_date).days) <= 30):
                        cancel = True
                        break
                if cancel:
                    continue
            
            row = {
                'Date': transaction['valueDate'],
                'Type': '',
                'Value': round(abs(transaction['amountInChf']), 8),
                'Security Name': '',
                'Transaction Currency': 'CHF',
                'Shares': '',
                'Exchange Rate': ''
            }
            
            if transaction['type'] == 'CONTRIBUTION':
                row['Type'] = 'Deposit'
            elif transaction['type'] == 'DIVIDEND':
                row['Type'] = 'Dividend'
                row['Security Name'] = transaction.get('description', '')
                row['Exchange Rate'] = last_ex_rate[row['Security Name']] # take from last time
                if row['Exchange Rate'] != '':
                    row['Gross Amount'] = row['Value']
                    row['Value'] = row['Value']*float(row['Exchange Rate']) 
                    row['Transaction Currency'] = ''
                    row['Currency Gross Amount'] = 'CHF'
                
            elif transaction['type'] in ['TRADE_SELL', 'TRADE_BUY']:
                row['Security Name'] = transaction.get('description', '')
                try:
                    shares, exchange_rate, isin, currency = extract_shares_and_exchange_rate(transaction['documentNumber'])
                    if isin not in securities:
                        securities[isin] = (row['Security Name'], currency)
                    row['Shares'] = shares
                    row['Exchange Rate'] = exchange_rate
                    last_ex_rate[row['Security Name']] = exchange_rate
                except FileNotFoundError as e:
                    print(f"Warning: PDF not found for transaction:")
                    print(f"{transaction['type'].split('_')[1]} {transaction['valueDate']} {transaction.get('description', '')} of CHF {transaction['amountInChf']}")
                if transaction['type'] == 'TRADE_SELL':
                    row['Type'] = 'Sell'
                else: # transaction['type'] == 'TRADE_BUY':
                    row['Type'] = 'Buy'
                
            elif transaction['type'] == 'INTEREST':
                row['Type'] = 'Interest'
            elif transaction['type'] == 'FEE_CHARGE':
                row['Type'] = 'Fees'
            
            writer.writerow(row)
            


# Process each account in the JSON data
securities = {}
for account_id, transactions in data['transactions'].items():
    process_transactions(account_id, transactions, securities)
    
with open("securities3.csv", mode='w', newline='') as file:
    writer = csv.writer(file)
    # Write the header
    writer.writerow(["ISIN", "Security Name", "Currency"])
    # Write the data
    for key, value in securities.items():
        writer.writerow([key, value[0], value[1]])

print("CSV files have been generated for each account.")

CSV files have been generated for each account.


In [72]:
import csv
from datetime import datetime

last_ex_rate = {}
last_curr = {}

def process_transactions(account_id, transactions, securities):
    # Prepare the CSV file names
    portfolio_csv_file_name = f"{account_id}_PortfolioTransaction.csv"
    account_csv_file_name = f"{account_id}_AccountTransaction.csv"
    
    # Define fieldnames for each CSV file
    portfolio_fieldnames = ['Date', 'Type', 'Value', 'Security Name', 'Transaction Currency', 'Shares', 'Exchange Rate']
    account_fieldnames = ['Date', 'Type', 'Value', 'Security Name', 'Transaction Currency', 'Shares', 'Exchange Rate', 'Gross Amount', 'Currency Gross Amount']
    
    # Open the CSV files for writing
    with open(portfolio_csv_file_name, 'w', newline='') as portfolio_csvfile, open(account_csv_file_name, 'w', newline='') as account_csvfile:
        portfolio_writer = csv.DictWriter(portfolio_csvfile, fieldnames=portfolio_fieldnames)
        account_writer = csv.DictWriter(account_csvfile, fieldnames=account_fieldnames)
        
        # Write the headers
        portfolio_writer.writeheader()
        account_writer.writeheader()
        
        # Track dividend cancellations to ignore corresponding dividends
        dividend_cancellations = []
        
        # First pass: build the list of dividend cancellations
        for transaction in transactions:
            if transaction['type'] == 'DIVIDEND_CANCELLATION':
                dividend_cancellations.append(transaction)
        
        # Second pass: process transactions and write to the appropriate CSV
        for transaction in transactions[::-1]:
            if transaction['type'] == 'DIVIDEND_CANCELLATION':
                continue
            
            if transaction['type'] == 'DIVIDEND':
                # Check if there is a matching DIVIDEND_CANCELLATION within 30 days
                cancel = False
                for cancel_transaction in dividend_cancellations:
                    cancel_date = datetime.strptime(cancel_transaction['valueDate'], '%Y-%m-%d')
                    transaction_date = datetime.strptime(transaction['valueDate'], '%Y-%m-%d')
                    if (cancel_transaction['amountInChf'] == transaction['amountInChf'] and
                        abs((transaction_date - cancel_date).days) <= 30):
                        cancel = True
                        break
                if cancel:
                    continue
            
            row = {
                'Date': transaction['valueDate'],
                'Type': '',
                'Value': round(abs(transaction['amountInChf']), 8),
                'Security Name': '',
                'Transaction Currency': 'CHF',
                'Shares': '',
                'Exchange Rate': ''
            }
            
            if transaction['type'] == 'CONTRIBUTION':
                row['Type'] = 'Deposit'
            elif transaction['type'] == 'DIVIDEND':
                row['Type'] = 'Dividend'
                row['Security Name'] = transaction.get('description', '')
                row['Exchange Rate'] = last_ex_rate[row['Security Name']] # take from last time
                if row['Exchange Rate'] != '':
                    # row['Gross Amount'] = row['Value']
                    # row['Gross Amount'] = row['Value']*float(row['Exchange Rate'])
                    # row['Value'] = row['Value']*float(row['Exchange Rate'])
                    row['Value'] = row['Value']
                    # row['Transaction Currency'] = last_curr[row['Security Name']]
                    row['Transaction Currency'] = 'CHF'
                    row['Currency Gross Amount'] = last_curr[row['Security Name']]
                
            elif transaction['type'] in ['TRADE_SELL', 'TRADE_BUY']:
                row['Security Name'] = transaction.get('description', '')
                try:
                    shares, exchange_rate, isin, currency = extract_shares_and_exchange_rate(transaction['documentNumber'])
                    if isin not in securities:
                        securities[isin] = (row['Security Name'], currency)
                    row['Shares'] = shares
                    row['Exchange Rate'] = exchange_rate
                    last_ex_rate[row['Security Name']] = exchange_rate
                    last_curr[row['Security Name']] = currency
                except FileNotFoundError as e:
                    print(f"Warning: PDF not found for transaction:")
                    print(f"{transaction['type'].split('_')[1]} {transaction['valueDate']} {transaction.get('description', '')} of CHF {transaction['amountInChf']}")
                if transaction['type'] == 'TRADE_SELL':
                    row['Type'] = 'Sell'
                else: # transaction['type'] == 'TRADE_BUY':
                    row['Type'] = 'Buy'
                
            elif transaction['type'] == 'INTEREST':
                row['Type'] = 'Interest'
            elif transaction['type'] == 'FEE_CHARGE':
                row['Type'] = 'Fees'
            
            # Write to the appropriate CSV file
            if transaction['type'] in ['TRADE_SELL', 'TRADE_BUY']:
                portfolio_writer.writerow(row)
            else:
                account_writer.writerow(row)
                
# Process each account in the JSON data
securities = {}
for account_id, transactions in data['transactions'].items():
    process_transactions(account_id, transactions, securities)
    
with open("securities3.csv", mode='w', newline='') as file:
    writer = csv.writer(file)
    # Write the header
    writer.writerow(["ISIN", "Security Name", "Currency"])
    # Write the data
    for key, value in securities.items():
        writer.writerow([key, value[0], value[1]])

print("CSV files have been generated for each account.")

CSV files have been generated for each account.


In [57]:
import re

def extract_currency(text):
    # Regular expression to match the pattern "Exchange rate CHF/USD 0.95128" or "Umrechnungskurs CHF/CAD \n0.65373\n"
    pattern = r'(?:Exchange rate|Umrechnungskurs) [A-Z]{3}/([A-Z]{3})\s*\d+\.\d+'
    
    # Search for the pattern in the text
    match = re.search(pattern, text, re.DOTALL)
    
    # If a match is found, return the currency
    if match:
        return match.group(1)
    else:
        return None

# Example usage
text1 = "Some long string with Exchange rate CHF/USD 0.95128 and some other text."
text2 = "Some long string with Umrechnungskurs CHF/CAD \n0.65373\n and some other text."
currency1 = extract_currency(text1)
currency2 = extract_currency(text2)
print(f"The currency from text1 is: {currency1}")
print(f"The currency from text2 is: {currency2}")



The currency from text1 is: USD
The currency from text2 is: CAD


'1.12629101'

In [42]:
shares = ''
exchange_rate = ''

try:
    # Open the PDF file
    # pdf_document = fitz.open("pdfs/21V-KGX-3RT.pdf")
    pdf_document = fitz.open("pdfs/21V-KQA-GV9.pdf")

    # Iterate through each page, even though there is only 1 usually
    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)
        text = page.get_text()

        # Search for shares value
        shares_match = re.search(r'(\d+\.\d+)\s+Anteile', text)
        if shares_match:
            shares = shares_match.group(1)

        # Search for exchange rate value
        exchange_rate_match = re.search(r'Umrechnungskurs\s+[\w/]+\s+(\d+\.\d+)', text)
        if exchange_rate_match:
            exchange_rate = exchange_rate_match.group(1)

        # Search for ISIN
        isin_match = re.search(r'ISIN.{0,8}([A-Z0-9]{12})', text, re.DOTALL)
        if isin_match:
            isin = isin_match.group(1)
    pdf_document.close()
except Exception as e:
    print(f"Error reading PDF {pdf_path}: {e}")
print(exchange_rate)
text

0.88787


"Terzo Vorsorgestiftung der WIR Bank\nAuberg 1\n4002 Basel\nE-Mail\ninfo@viac.ch\nTelefon 0800 80 40 40\nwww.viac.ch\nVertrag \n3.172.513.674\nPortfolio 3.172.513.674.01\nHerr\nManuel Meier\nGeissbergstrasse 33\n5408 Ennetbaden\nBasel, 14.09.2023\nBörsenabrechnung - Verkauf iShares Nasdaq 100\nWir haben für Sie folgenden Auftrag ausgeführt:\nS. E. & O.\nFreundliche Grüsse\nTerzo Vorsorgestiftung\nAnzeige ohne Unterschrift\nOrder:\nVerkauf\n1.531 Anteile iShares Nasdaq 100\nISIN: \nIE00B53SZB19\nKurs:\nUSD 878.00\nBetrag\nUSD\n1'344.30\nUmrechnungskurs CHF/USD 0.88787\nCHF\n1'193.56\nStempelsteuer\nCHF\n-0.56\nVerrechneter Betrag:\nValuta 14.09.2023\nCHF\n1'193.00\n"

In [34]:

def extract_isin(text):
    # Regular expression to match the pattern "ISIN" followed by at most 8 other symbols (including newlines) and then a 12 digit alphanumeric string
    pattern = r'ISIN.{0,8}([A-Z0-9]{12})'
    
    # Search for the pattern in the text
    match = re.search(pattern, text, re.DOTALL)
    
    # If a match is found, return the ISIN
    if match:
        return match.group(1)
    else:
        return None

# Example usage
text = "Some long string with \nISIN: \nLU0439730705\n and some other text."
isin = extract_isin(text)
print(f"The ISIN is: {isin}")

The ISIN is: LU0439730705


In [24]:
shares = ''
exchange_rate = ''

document_number = '21V-KZW-TLG'
# Find the PDF file that contains the document number as a substring
pdf_folder = 'pdfs'
pdf_files = [f for f in os.listdir(pdf_folder) if document_number in f]

if not pdf_files:
    raise FileNotFoundError(f"No PDF file found containing document number {document_number}")

pdf_path = os.path.join(pdf_folder, pdf_files[0])

# Open the PDF file
pdf_document = fitz.open(pdf_path)

# Iterate through each page
for page_num in range(len(pdf_document)):
    page = pdf_document.load_page(page_num)
    text = page.get_text()

    # Search for shares value
    shares_match = re.search(r'(?:Kauf|Buy)\n(\d+\.\d+)\n', text)
    if shares_match:
        shares = shares_match.group(1)
    if len(shares)<2:
        print(f"Error in transaction {document_number}. File exists but could not find number of shares!")


    # Search for exchange rate value
    exchange_rate_match = re.search(r'(?:Exchange rate|Umrechnungskurs) [A-Z]{3}/[A-Z]{3} (\d+\.\d+)\n', text)
    if exchange_rate_match:
        exchange_rate = exchange_rate_match.group(1)

pdf_document.close()
text

Error in transaction 21V-KZW-TLG. File exists but could not find number of shares!


'Terzo Vorsorgestiftung der WIR Bank\nAuberg 1\n4002 Basel\nE-Mail\ninfo@viac.ch\nTelefon 0800 80 40 40\nwww.viac.ch\nVertrag \n3.172.513.674\nPortfolio 3.172.513.674.01\nHerr\nManuel Meier\nHohestrasse 11\n5107 Schinznach Dorf\nBasel, 06.12.2024\nBörsenabrechnung - Kauf Swisscanto SMI (SPI 20)\nWir haben für Sie folgenden Auftrag ausgeführt:\nS. E. & O.\nFreundliche Grüsse\nTerzo Vorsorgestiftung\nAnzeige ohne Unterschrift\nOrder:\nKauf\n2.274 Anteile Swisscanto SMI (SPI 20)\nISIN: \nCH0215804714\nKurs:\nCHF 166.72\nBetrag\nCHF\n379.17\nVerrechneter Betrag:\nValuta 05.12.2024\nCHF\n379.17\n'

In [14]:
text

"Terzo Vorsorgestiftung der WIR Bank\nAuberg 1\n4002 Basel\nInternet\nwww.viac.ch\nEmail\ninfo@viac.ch\nPhone\n0800 80 40 40\nContract\n3.172.513.674\nPortfolio\n3.172.513.674.01\nBasel, 03.07.2020\nExchange Settlement - Buy\nWe have executed the following order for you:\nOrder:\nBuy\n0.168\nQty\nCSIF SPI Extra\nISIN: \nCH0110869143\nPrice:\nCHF 2'042.94\nAmount\nCHF \n342.37\n \nCharged amount:\nValue date 03.07.2020\nCHF \n342.37\nS. E. & O.\nKind regards\nTerzo Pension Foundation\nDisplay without signature\nMr\nManuel Meier\nSt. Bernhardstrasse 33\n5430 Wettingen\n"

In [7]:
import os
import re
import fitz  # PyMuPDF

def remove_duplicates_and_concatenate_pdfs(folder_path):
    # Get list of all files in the folder
    files = os.listdir(folder_path)
    
    # Filter out only PDF files
    pdf_files = [f for f in files if f.endswith('.pdf')]
    
    # Dictionary to keep track of unique files
    unique_files = {}
    
    # Regular expression to match duplicate files
    duplicate_pattern = re.compile(r'\((\d+)\)\.pdf$')
    
    for pdf in pdf_files:
        # Remove the duplicate pattern from the filename
        base_name = re.sub(duplicate_pattern, '.pdf', pdf)
        
        # If the base name is not in the dictionary, add it
        if base_name not in unique_files:
            unique_files[base_name] = pdf
    
    # List to store the paths of unique PDF files
    unique_pdf_paths = [os.path.join(folder_path, unique_files[base_name]) for base_name in unique_files]

    
    # Create a new PDF document to concatenate all PDFs
    output_pdf = fitz.open()
    
    for pdf_path in unique_pdf_paths:
        # Open each PDF file
        input_pdf = fitz.open(pdf_path)
        
        # Append each page to the output PDF
        for page_num in range(len(input_pdf)):
            output_pdf.insert_pdf(input_pdf, from_page=page_num, to_page=page_num)
    
    # Save the concatenated PDF
    output_pdf.save(os.path.join(folder_path, 'concatenated_output.pdf'))
    
    print(f"Concatenated PDF saved as 'concatenated_output3.pdf' in {folder_path}")
    return unique_pdf_paths

# Folder path containing the PDF files
folder_path = r"C:\Users\Manuel Meier\Downloads\viac1transactions"

# Call the function to remove duplicates and concatenate PDFs
remove_duplicates_and_concatenate_pdfs(folder_path)

Concatenated PDF saved as 'concatenated_output3.pdf' in C:\Users\Manuel Meier\Downloads\viac1transactions


['C:\\Users\\Manuel Meier\\Downloads\\viac1transactions\\21V-KAK-29H.pdf',
 'C:\\Users\\Manuel Meier\\Downloads\\viac1transactions\\21V-KAK-L68(1).pdf',
 'C:\\Users\\Manuel Meier\\Downloads\\viac1transactions\\21V-KAK-NB9(1).pdf',
 'C:\\Users\\Manuel Meier\\Downloads\\viac1transactions\\21V-KAK-QS0.pdf',
 'C:\\Users\\Manuel Meier\\Downloads\\viac1transactions\\21V-KAK-RRL(1).pdf',
 'C:\\Users\\Manuel Meier\\Downloads\\viac1transactions\\21V-KAK-W06(1).pdf',
 'C:\\Users\\Manuel Meier\\Downloads\\viac1transactions\\21V-KAK-X5R(1).pdf',
 'C:\\Users\\Manuel Meier\\Downloads\\viac1transactions\\21V-KAK-Y7T(1).pdf',
 'C:\\Users\\Manuel Meier\\Downloads\\viac1transactions\\21V-KAL-6X7(1).pdf',
 'C:\\Users\\Manuel Meier\\Downloads\\viac1transactions\\21V-KAL-9QV(1).pdf',
 'C:\\Users\\Manuel Meier\\Downloads\\viac1transactions\\21V-KAL-ALB(1).pdf',
 'C:\\Users\\Manuel Meier\\Downloads\\viac1transactions\\21V-KAL-NBW(1).pdf',
 'C:\\Users\\Manuel Meier\\Downloads\\viac1transactions\\21V-KAL-O7E(1

In [5]:
# Regular expression to match duplicate files
duplicate_pattern = re.compile(r'\((\d+)\)\.pdf$')

base_name = re.sub(duplicate_pattern, '.pdf', 'C:\\Users\\Manuel Meier\\Downloads\\viac1transactions\\21V-KCE-D9V(2).pdf')
print(base_name)

C:\Users\Manuel Meier\Downloads\viac1transactions\21V-KCE-D9V.pdf
